In [ ]:
from util import generate_simple_prompts, initialize_df, split_dataset
import pandas as pd
import random
from datasets import Dataset, DatasetDict
from transformers import set_seed, AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig, TrainingArguments
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from pprint import pprint

In [2]:
SEED = 1212

TRAIN = 0.85
TEST = 0.1
VALIDATION = 0.05

OUTPUT_DIR = ""


In [3]:
set_seed(SEED)

In [4]:
df = initialize_df()

Constructed dataframe found. Reading source...
Non-empty dataframe read.


In [5]:
prompts = generate_simple_prompts(df, shots=1, fuzzy=True) + generate_simple_prompts(df, shots=1, fuzzy=True)
random.shuffle(prompts)

In [6]:
print(type(prompts))
print(prompts[0], "\n")
print(prompts[-1])

<class 'list'>
English: "Never had a reason to celebrate it," Devyn said, shrugging it off as he handed over two twenties to the cashier dressed as an elf.
French: Je n'ai jamais eu de raison de le célébrer, déclara Devyn, en tendant deux billets de vingt à la caissière déguisée en elfe.
English: "Yes," Devyn said, shrugging it off since he hadn't been left with much of a choice.
French: Oui, dit Devyn en balayant le sujet puisqu'il n'avait pas eu trop le choix.
 

English: "As long as the surge doesn't get too much bigger than they predict, we'll be okay.
French: Tant que l'onde de tempête n'est pas plus forte que ce qu'ils prévoient, tout ira bien.
English: As long as she didn't want to go onto the beach, he'd be fine.
French: Tant qu'elle ne voulait pas aller sur la plage, il s'en sortirait.



In [7]:
prompts = [{"text": prompt} for prompt in prompts]
dataset = Dataset.from_list(prompts)
dataset = split_dataset(dataset, train=TRAIN, test=TEST, validation=VALIDATION)
pprint(dataset)

{'test': Dataset({
    features: ['text'],
    num_rows: 2249
}),
 'train': Dataset({
    features: ['text'],
    num_rows: 19048
}),
 'validation': Dataset({
    features: ['text'],
    num_rows: 1191
})}


In [8]:
pprint(dataset['train'][4])

{'text': 'English: For those not familiar with these massive stormswhen the '
         'eye is over you, the worst is yet to come.\n'
         'French: Pour ceux qui ne sont pas familiers avec ces énormes '
         "tempêtes, lorsque l'œil est au-dessus de vous, le pire est à venir.\n"
         'English: Worries about the impending storm pushed the memories '
         'closer to the surface, and his body felt the toll.\n'
         'French: Les inquiétudes liées à la tempête imminente faisaient '
         'remonter les souvenirs à la surface, et son corps en ressentait les '
         'effets.\n'}
